 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8887/notebooks/keras/presentation_nb.ipynb#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="http://localhost:8887/notebooks/keras/presentation_nb.ipynb#Train-a-fully-connected-model" data-toc-modified-id="Train-a-fully-connected-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Train a fully connected model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8887/notebooks/keras/presentation_nb.ipynb#Load-the-datasets" data-toc-modified-id="Load-the-datasets-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load the datasets</a></span></li><li><span><a href="http://localhost:8887/notebooks/keras/presentation_nb.ipynb#Set-up-the-model" data-toc-modified-id="Set-up-the-model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Set up the model</a></span></li></ul></li><li><span><a href="http://localhost:8887/notebooks/keras/presentation_nb.ipynb#Convolutional-model" data-toc-modified-id="Convolutional-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Convolutional model</a></span></li></ul></div>

# CBRAIN experiments with Keras, etc.

ToDo:
- Load data
- Run a simple fully connected net to get close to 0.7 R2
- Visualization of output
    - some examples
    - checkout the biases, etc
- low level implementation in python

## Setup

Basic imports and path definitions. The data loading, keras models and other utility functions are imported from scripts in this directory. I am running this interactively on a 4GB GPU. Note that the CPU RAM might need to be bigger due to the large data sets.

In [1]:
# Imports 
from importlib import reload
import utils; reload(utils); from utils import *
import data_generator; reload(data_generator); from data_generator import *
import models; reload(models); from models import *

Using TensorFlow backend.


In [2]:
# Limit Tensorflow GPU memory usage. 
# Note that apparently it's not possible to change the allocation or release memory again.
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5   # GPU RAM usage fraction of 4GB
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
# Plotting setup
%matplotlib inline
np.random.seed(42)
sns.set_style('dark')
sns.set_palette('deep')
sns.set_context('talk')
plt.rcParams["figure.figsize"] = (10,7)

In [4]:
# Define data paths
data_dir = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/preprocessed_data/detailed_files/' # Full dataset
%ls $data_dir

SPCAM_mean_detailed.nc              SPCAM_outputs_train_by_lon_flat.nc
SPCAM_outputs_detailed.nc           SPCAM_outputs_train_by_lon.nc
SPCAM_outputs_flat.nc               SPCAM_outputs_valid_by_lon_flat.nc
SPCAM_outputs_flat_train_random.nc  SPCAM_outputs_valid_by_lon.nc
SPCAM_outputs_flat_valid_random.nc  SPCAM_std_detailed.nc


In [5]:
# Define a dictionary containing the feature and target variables
# and the number of dimensions
feature_vars = OrderedDict({
    'TAP': 2,             # Temperature [z, sample]
    'QAP': 2,             # Specific humidity [z, sample]
    'OMEGA': 2,           # [z, sample]
    'dTdt_adiabatic': 2,  # [z, sample]
    'dQdt_adiabatic': 2,  # [z, sample]
    'QRL': 2,             # Long wave heating rate [z, sample]
    'QRS': 2,             # Short wave heating rate [z, sample]
    'SHFLX': 1,           # [sample]
    'LHFLX': 1,           # [sample]
    'LAT': 1,             # Latitude [sample]
})
target_vars = OrderedDict({
    'SPDT': 2,            # SP temperature tendency [z, sample]
    'SPDQ': 2,            # SP humidity tendency [z, sample]
})

## Train a fully connected model

Here, we will train a fully connected neural net which we can then use for visualization and as a basis for a low-level implementation.

### Load the datasets

I am using the train and validation data I created with the function in `data_processing`. The loading of the data to be used in the NN is done with the `DataSet` function from `data_generator.py`. For now I am using the randomly separated data set since this is what I believe is closest to the data sets you guys are using.

In [7]:
train_set = DataSet(data_dir, 'SPCAM_outputs_flat_train_random.nc', 'SPCAM_mean_detailed.nc',
                    'SPCAM_std_detailed.nc', feature_vars.keys(), flat_input=True)
valid_set = DataSet(data_dir, 'SPCAM_outputs_flat_valid_random.nc', 'SPCAM_mean_detailed.nc',
                    'SPCAM_std_detailed.nc', feature_vars.keys(), flat_input=True)

In [9]:
train_set.features.shape, valid_set.features.shape

((3696230, 150), (924058, 150))

### Set up the model

The actual model setup is in `models.py` and the loss functions are in `losses.py`.

In [20]:
model = fc_model(train_set.features.shape[1], valid_set.targets.shape[1], 
                 hidden_layers=[1024, 512], 
                 lr=0.0001, 
                 loss='mae')

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 1024)              154624    
_________________________________________________________________
dense_14 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_15 (Dense)             (None, 42)                21546     
Total params: 700,970
Trainable params: 700,970
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_set.features, train_set.targets, batch_size=512, epochs=10, 
          validation_data=(valid_set.features, valid_set.targets))

Train on 3696230 samples, validate on 924058 samples
Epoch 1/10
3696230/3696230 [==============================] - 75s - loss: 0.0201 - rmse: 0.0363 - log_loss: -1.5928 - total_error: 90.5786 - unexplained_error: 57.5901 - rsquared: 0.3640 - total_error_avgAx0: 78.6072 - rsquared_avgAx0: 0.2663 - val_loss: 0.0173 - val_rmse: 0.0321 - val_log_loss: -1.6448 - val_total_error: 90.1702 - val_unexplained_error: 43.5502 - val_rsquared: 0.5164 - val_total_error_avgAx0: 78.2253 - val_rsquared_avgAx0: 0.4420
Epoch 2/10
3696230/3696230 [==============================] - 73s - loss: 0.0170 - rmse: 0.0315 - log_loss: -1.6549 - total_error: 90.5778 - unexplained_error: 42.1389 - rsquared: 0.5341 - total_error_avgAx0: 78.6054 - rsquared_avgAx0: 0.4626 - val_loss: 0.0168 - val_rmse: 0.0311 - val_log_loss: -1.6590 - val_total_error: 90.1702 - val_unexplained_error: 41.1003 - val_rsquared: 0.5437 - val_total_error_avgAx0: 78.2253 - val_rsquared_avgAx0: 0.4734
Epoch 3/10
3696230/3696230 [===============

KeyboardInterrupt: 

In [23]:
model2 = fc_model(train_set.features.shape[1], valid_set.targets.shape[1], 
                 hidden_layers=[1024, 512, 512], 
                 lr=0.0001, 
                 loss='mae')
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 1024)              154624    
_________________________________________________________________
dense_17 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_18 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_19 (Dense)             (None, 42)                21546     
Total params: 963,626
Trainable params: 963,626
Non-trainable params: 0
_________________________________________________________________


In [26]:
model2.fit(train_set.features, train_set.targets, batch_size=512, epochs=5, 
          validation_data=(valid_set.features, valid_set.targets))

Train on 3696230 samples, validate on 924058 samples
Epoch 1/5
3696230/3696230 [==============================] - 87s - loss: 0.0147 - rmse: 0.0270 - log_loss: -1.7449 - total_error: 90.5777 - unexplained_error: 33.6344 - rsquared: 0.6283 - total_error_avgAx0: 78.6071 - rsquared_avgAx0: 0.5712 - val_loss: 0.0147 - val_rmse: 0.0270 - val_log_loss: -1.7431 - val_total_error: 90.1702 - val_unexplained_error: 33.4928 - val_rsquared: 0.6279 - val_total_error_avgAx0: 78.2253 - val_rsquared_avgAx0: 0.5706
Epoch 2/5
3696230/3696230 [==============================] - 87s - loss: 0.0146 - rmse: 0.0269 - log_loss: -1.7473 - total_error: 90.5788 - unexplained_error: 33.3408 - rsquared: 0.6313 - total_error_avgAx0: 78.6050 - rsquared_avgAx0: 0.5747 - val_loss: 0.0146 - val_rmse: 0.0269 - val_log_loss: -1.7474 - val_total_error: 90.1702 - val_unexplained_error: 33.6649 - val_rsquared: 0.6264 - val_total_error_avgAx0: 78.2253 - val_rsquared_avgAx0: 0.5689
Epoch 3/5
3696230/3696230 [==================

In [ ]:
model2.save()

## Convolutional model

In [13]:
train_set_cov = DataSet(data_dir, 'SPCAM_outputs_flat_train_random.nc', 'SPCAM_mean_detailed.nc',
                    'SPCAM_std_detailed.nc', feature_vars.keys(), flat_input=True,
                       convolution=True)
valid_set_cov = DataSet(data_dir, 'SPCAM_outputs_flat_valid_random.nc', 'SPCAM_mean_detailed.nc',
                    'SPCAM_std_detailed.nc', feature_vars.keys(), flat_input=True,
                       convolution=True)

In [14]:
train_set_cov.features[0].shape

(3696230, 21, 7)

In [16]:
model_cov = conv_model(train_set_cov.features[0].shape[1:], train_set_cov.features[1].shape[1],
                   train_set_cov.targets.shape[1], [32, 32, 32, 32, 32, 32], [], 1e-3, 'mae')

In [17]:
model_cov.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 21, 7)         0                                            
____________________________________________________________________________________________________
conv1d_7 (Conv1D)                (None, 21, 32)        704         input_3[0][0]                    
____________________________________________________________________________________________________
conv1d_8 (Conv1D)                (None, 21, 32)        3104        conv1d_7[0][0]                   
____________________________________________________________________________________________________
conv1d_9 (Conv1D)                (None, 21, 32)        3104        conv1d_8[0][0]                   
___________________________________________________________________________________________